### Objective:
Given a list of wallet addresses, fetch their transaction history from the Compound V2/V3 protocol and assign a **risk score from 0 to 1000** to each wallet based on their behavior.

### Deliverables:
- A CSV with columns: `wallet_id`, `score`
- A write-up explaining:
  - Data collection process
  - Feature selection rationale
  - Scoring methodology
  - Risk indicators used

## Import libraries, set the path and load the wallet IDs

In [5]:
import pandas as pd
import numpy as np
import requests
import json
from tqdm import tqdm
import time
import os

wallet_path = r"C:\Users\Poornima Kc\Documents\zeru_wallet_risk\wallet id.xlsx"  

wallets_df = pd.read_excel(wallet_path)
wallets_df.columns = ['wallet_id']  

wallet_list = wallets_df['wallet_id'].tolist()

wallets_df.head()

,wallet_id
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3
1,0x06b51c6882b27cb05e712185531c1f74996dd988
2,0x0795732aacc448030ef374374eaae57d2965c16c
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae


### Fetch Transaction History from Compound V2/V3

We will now fetch historical DeFi transactions for each wallet using an API like:
- **Covalent API**
- **DeFiLlama**
- **Etherscan (backup)**

Our goal is to extract only **Compound V2 or V3** protocol interactions for scoring.


In [6]:
import requests

COVALENT_API_KEY = "cqt_rQY8f9WtCd3fkcQdwBgD8x8PY8mr"

def fetch_wallet_transactions(wallet):
    url = f"https://api.covalenthq.com/v1/1/address/{wallet}/transactions_v2/?key={COVALENT_API_KEY}"
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"⚠️ Error fetching for {wallet}: {response.status_code}")
            return None
    except Exception as e:
        print(f"❌ Exception occurred for {wallet}: {e}")
        return None

###  Feature Engineering

We extract features like:
- Total number of Compound txns
- Total borrowed and repaid
- Total liquidations
- Frequency of lending vs borrowing
- Time active (days between first and last txn)


In [7]:
def extract_features(txns):
    num_borrows = sum(1 for tx in txns if 'borrow' in str(tx).lower())
    num_repays = sum(1 for tx in txns if 'repay' in str(tx).lower())
    num_liquidations = sum(1 for tx in txns if 'liquidation' in str(tx).lower())
    
    score = (num_repays - num_borrows + 1) * 100  # Dummy logic

    return {
        'num_borrows': num_borrows,
        'num_repays': num_repays,
        'num_liquidations': num_liquidations,
        'score': max(0, min(1000, score))  # Clamp between 0 and 1000
    }

### Scoring Explanation

The wallet risk score is calculated based on:

- **Borrow Activity**: Higher borrows indicate potential risk.
- **Repayment Count**: Indicates responsible behavior.
- **Liquidation Events**: Very risky signal.
- **Duration of Activity**: Long active wallets considered safer.

Final score is scaled to 0–1000 using a normalized rule-based formula.

In [8]:
wallet_scores = {}

for wallet in tqdm(wallet_list):
    tx_data = fetch_wallet_transactions(wallet)
    
    if tx_data and "data" in tx_data and tx_data["data"].get("items"):
        items = tx_data["data"]["items"]

        features = extract_features(items)
        wallet_scores[wallet] = features['score']
    else:
        print(f"❌ Skipping {wallet} due to missing or invalid data")
        wallet_scores[wallet] = 0  # or None

    time.sleep(0.3)  # respect rate limits


 73%|██████████████████████████████████████████████████████████▉                      | 75/103 [09:17<03:17,  7.06s/it]

❌ Exception occurred for 0xa7f3c74f0255796fd5d3ddcf88db769f7a6bf46a: ('Connection broken: IncompleteRead(2706 bytes read, 7534 more expected)', IncompleteRead(2706 bytes read, 7534 more expected))
❌ Skipping 0xa7f3c74f0255796fd5d3ddcf88db769f7a6bf46a due to missing or invalid data


100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [14:53<00:00,  8.67s/it]


In [9]:
# Save results to CSV
pd.DataFrame(wallet_scores.items(), columns=["wallet_id", "score"]).to_csv("wallet_risk_scores.csv", index=False)